In [1]:
import json
import pickle
f=open('D:/COCO/annot_2017/instances_val2017.json','r')
annot_dict=json.load(f)
#annot.keys()

In [2]:
annotations=annot_dict['annotations']
images=annot_dict['images']

In [3]:
count=0
for annot in annotations:
    if annot['iscrowd']>0:
        count+=1
        #print(annot['iscrowd'])
print(count)

446


In [4]:
annotations[1]

{'segmentation': [[289.74,
   443.39,
   302.29,
   445.32,
   308.09,
   427.94,
   310.02,
   416.35,
   304.23,
   405.73,
   300.14,
   385.01,
   298.23,
   359.52,
   295.04,
   365.89,
   282.3,
   362.71,
   275.29,
   358.25,
   277.2,
   346.14,
   280.39,
   339.13,
   284.85,
   339.13,
   291.22,
   338.49,
   293.77,
   335.95,
   295.04,
   326.39,
   297.59,
   317.47,
   289.94,
   309.82,
   287.4,
   288.79,
   286.12,
   275.41,
   284.21,
   271.59,
   279.11,
   276.69,
   275.93,
   275.41,
   272.1,
   271.59,
   274.01,
   267.77,
   275.93,
   265.22,
   277.84,
   264.58,
   282.3,
   251.2,
   293.77,
   238.46,
   307.79,
   221.25,
   314.79,
   211.69,
   325.63,
   205.96,
   338.37,
   205.32,
   347.29,
   205.32,
   353.03,
   205.32,
   361.31,
   200.23,
   367.95,
   202.02,
   372.27,
   205.8,
   382.52,
   215.51,
   388.46,
   225.22,
   399.25,
   235.47,
   399.25,
   252.74,
   390.08,
   247.34,
   386.84,
   247.34,
   388.46,
   256.52,
 

In [5]:
_LABELS=[]
for cat in annot_dict['categories']:
    _LABELS.append(cat['name'])
f=open('D:/COCO/labels_coco','wb')
pickle.dump(_LABELS,f)
f.close()

In [6]:
LABELS={}
for cat in annot_dict['categories']:
    LABELS[cat['id']]=cat['name']

In [7]:
LABELS

{1: 'person',
 2: 'bicycle',
 3: 'car',
 4: 'motorcycle',
 5: 'airplane',
 6: 'bus',
 7: 'train',
 8: 'truck',
 9: 'boat',
 10: 'traffic light',
 11: 'fire hydrant',
 13: 'stop sign',
 14: 'parking meter',
 15: 'bench',
 16: 'bird',
 17: 'cat',
 18: 'dog',
 19: 'horse',
 20: 'sheep',
 21: 'cow',
 22: 'elephant',
 23: 'bear',
 24: 'zebra',
 25: 'giraffe',
 27: 'backpack',
 28: 'umbrella',
 31: 'handbag',
 32: 'tie',
 33: 'suitcase',
 34: 'frisbee',
 35: 'skis',
 36: 'snowboard',
 37: 'sports ball',
 38: 'kite',
 39: 'baseball bat',
 40: 'baseball glove',
 41: 'skateboard',
 42: 'surfboard',
 43: 'tennis racket',
 44: 'bottle',
 46: 'wine glass',
 47: 'cup',
 48: 'fork',
 49: 'knife',
 50: 'spoon',
 51: 'bowl',
 52: 'banana',
 53: 'apple',
 54: 'sandwich',
 55: 'orange',
 56: 'broccoli',
 57: 'carrot',
 58: 'hot dog',
 59: 'pizza',
 60: 'donut',
 61: 'cake',
 62: 'chair',
 63: 'couch',
 64: 'potted plant',
 65: 'bed',
 67: 'dining table',
 70: 'toilet',
 72: 'tv',
 73: 'laptop',
 74: 'mo

In [8]:
images_dict={}
for image in images:
    images_dict[image['id']]=[image['file_name'],image['width'],image['height']]
annots_dict={}
for annot in annotations:
    annots_dict[annot['image_id']]=[]
for annot in annotations:
    annots_dict[annot['image_id']].append(annot)

In [9]:
IMAGE_W=608
IMAGE_H=608
train_images={}
for _id in annots_dict.keys():
    train_img={}
    image=images_dict[_id]
    annots=annots_dict[_id]

    train_img['filename']=image[0]
    train_img['width']=image[1]
    train_img['height']=image[2]
    objs=[]
    wf=IMAGE_W/image[1]
    hf=IMAGE_H/image[2]
    for annot in annots:
        if annot['iscrowd']==0:
           obj={}
           bbox=annot['bbox']
           obj['name']=LABELS[annot['category_id']]
           obj['xmin']=bbox[0]*wf
           obj['ymin']=bbox[1]*hf
           obj['xmax']=(bbox[0]+bbox[2])*wf
           obj['ymax']=(bbox[1]+bbox[3])*hf
           objs.append(obj)
    train_img['object']=objs
    train_images[image[0]]=train_img

In [10]:
import glob
import pickle
fnames=[]  
train_data=[]
test_data=[]
i=0    
for file in glob.glob('D:/COCO/val2017/*'):
    i+=1
    file_name=file.split('\\')[-1]
    try:data=train_images[file_name]
    except:continue
    train_data.append(data)
    
i=0
for file in glob.glob('D:/COCO/val2017/*'):
    i+=1
    
    file_name=file.split('\\')[-1]
    try:data=train_images[file_name]
    except:continue
    test_data.append(data)
    
f=open('D:/COCO/train_coco_dfs','wb')
pickle.dump(train_data,f)
f.close()
f=open('D:/COCO/test_coco_dfs','wb')
pickle.dump(test_data,f)
f.close()

In [11]:
len(train_data)

4952

In [12]:
import glob
for file in glob.glob('D:/COCO/val2017/*'):
    print(file.split('\\')[-1])
    break

000000000139.jpg


### Extra functions

In [13]:
import csv

def read_data(dataset):
  f=open(dataset+'/label.csv')
  file=csv.reader(f,delimiter=',')
  data=[]
  i=0
  sc=416.0
  for line in file:
      dt=line
      H=dt[0]
      W=dt[1]
      xmin=(float(dt[2]))/sc
      ymin=(float(dt[3]))/sc
      xmax=(float(dt[4]))/sc
      ymax=(float(dt[5]))/sc

      output={
          'filename':dataset+'/images/'+str(i)+'.jpg',
          'height':H,
          'width':W,
          'object':[{'name':'None',
          'xmin':xmin*IMAGE_W,
          'ymin':ymin*IMAGE_H,
          'xmax':xmax*IMAGE_W,
          'ymax':ymax*IMAGE_H}]
          }

      data.append(output)
      i=i+1
  return data
